In [26]:
from transformers import pipeline
import pandas as pd

In [27]:
data_pd = pd.read_csv('Codage_CV versus HR_for Irene Berezin-Dec-17-2024.csv')

In [28]:
data_pd

,Question,ResponseM,pid
0,2,protecting the elderly protecting children rig...,1890243c-fec2-eb11-a7ad-28187818aecc
1,2,"amnesty international freedom of opinions, pre...",62e4e4eb-9bb9-eb11-a7ad-28187818aecc
2,2,liberté droit d'un individus droit familial,7b8260c7-9bb9-eb11-a7ad-28187818aecc
3,1,family nature canadian rights freedom financial,9e78e854-9bb9-eb11-a7ad-28187818aecc
4,2,liberté respecter les autres manger a sa faim ...,31d268bb-9bb9-eb11-a7ad-28187818aecc
...,...,...,...
877,1,nothing the outdoors not indigenous peoples,0461f6af-a1c3-eb11-a7ad-28187818aecc
878,1,anglais hydro sirop forêts hiver autochtones,5094395e-eebe-eb11-a7ad-28187818aecc
879,2,certain parties support keeping human rights f...,bc534d7e-fdc2-eb11-a7ad-28187818aecc
880,2,equality justice conflict going too far with n...,116391bc-eebe-eb11-a7ad-28187818aecc


In [29]:

classifier = pipeline("text-classification", model="papluca/xlm-roberta-base-language-detection")

def detect_language_transformer(text):
    result = classifier(text[:512], return_all_scores=True)  # Get all probabilities
    scores = {x['label']: x['score'] for x in result[0]}

    target_languages = ['fr', 'en']
    filtered_scores = {k: scores[k] for k in target_languages if k in scores}
    
    # model was labelling some french text as italian, so forcing it to label it as french
    if filtered_scores:
        return max(filtered_scores, key=filtered_scores.get)
    else:
        # Fallback: If neither 'fr' nor 'en' is detected, choose based on closest match
        if 'it' in scores and scores['it'] > 0.3:
            return 'fr' 
        return 'en' 

data_pd['language'] = data_pd['ResponseM'].dropna().apply(detect_language_transformer)




Device set to use cpu
c:\Users\irene\anaconda3\envs\sent\Lib\site-packages\transformers\pipelines\text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [30]:
data_pd

,Question,ResponseM,pid,language
0,2,protecting the elderly protecting children rig...,1890243c-fec2-eb11-a7ad-28187818aecc,en
1,2,"amnesty international freedom of opinions, pre...",62e4e4eb-9bb9-eb11-a7ad-28187818aecc,en
2,2,liberté droit d'un individus droit familial,7b8260c7-9bb9-eb11-a7ad-28187818aecc,fr
3,1,family nature canadian rights freedom financial,9e78e854-9bb9-eb11-a7ad-28187818aecc,en
4,2,liberté respecter les autres manger a sa faim ...,31d268bb-9bb9-eb11-a7ad-28187818aecc,fr
...,...,...,...,...
877,1,nothing the outdoors not indigenous peoples,0461f6af-a1c3-eb11-a7ad-28187818aecc,en
878,1,anglais hydro sirop forêts hiver autochtones,5094395e-eebe-eb11-a7ad-28187818aecc,fr
879,2,certain parties support keeping human rights f...,bc534d7e-fdc2-eb11-a7ad-28187818aecc,en
880,2,equality justice conflict going too far with n...,116391bc-eebe-eb11-a7ad-28187818aecc,en


In [31]:
french = data_pd[data_pd['language'] == 'fr']
french

,Question,ResponseM,pid,language
2,2,liberté droit d'un individus droit familial,7b8260c7-9bb9-eb11-a7ad-28187818aecc,fr
4,2,liberté respecter les autres manger a sa faim ...,31d268bb-9bb9-eb11-a7ad-28187818aecc,fr
5,2,égalité respect liberté famille s’exprimer etr...,25ae2752-eebe-eb11-a7ad-28187818aecc,fr
6,1,famille democracie respecter equitable securit...,a92a5edf-9bb9-eb11-a7ad-28187818aecc,fr
7,1,respect gentillesse honnetete integrite famill...,9f5d9e78-eebe-eb11-a7ad-28187818aecc,fr
...,...,...,...,...
864,2,privilège acquis inégalité société combat espoir,80eccb65-eebe-eb11-a7ad-28187818aecc,fr
870,2,ethnicité egalité esprit de partage différence...,ed215672-fec2-eb11-a7ad-28187818aecc,fr
872,2,bafoués nécessaires charte conventions autocht...,817ad272-9bb9-eb11-a7ad-28187818aecc,fr
874,2,iniquité afrique femmes famine pauvreté égoïsme,1f6191bc-eebe-eb11-a7ad-28187818aecc,fr


In [32]:
label_map = {
    "LABEL_0": "negative",  
    "LABEL_1": "neutral",
    "LABEL_2": "positive",
    "negative": "negative", 
    "neutral":  "neutral",
    "positive": "positive"
}

In [33]:
#initializing pipelines
sentiment_pipeline_en = pipeline(
    "sentiment-analysis",
    model="cardiffnlp/twitter-roberta-base-sentiment-latest", use_fast=False,return_all_scores=True,
    framework="pt"
)

sentiment_pipeline_fr = pipeline(
    "sentiment-analysis",
    model="cardiffnlp/twitter-xlm-roberta-base-sentiment",use_fast=False,return_all_scores=True,
    framework="pt"
)

#preprocessing
english_data = data_pd[data_pd['language'] == 'en']['ResponseM'].dropna().astype(str).to_list()
french_data = data_pd[data_pd['language'] == 'fr']['ResponseM'].dropna().astype(str).to_list()

batch_size = 32
results_en = []
results_fr = []

for i in range(0, len(english_data), batch_size):
    batch = english_data[i:i + batch_size]
    batch_results = sentiment_pipeline_en(batch)
    results_en.extend(batch_results)
for i in range(0, len(french_data), batch_size):
    batch = french_data[i:i + batch_size]
    batch_results = sentiment_pipeline_fr(batch)
    results_fr.extend(batch_results)

all_scores_en = []
for doc_result in results_en:  
    row = {"negative": 0.0, "neutral": 0.0, "positive": 0.0}
    for label_score in doc_result:
        raw_label = label_score["label"]       
        score = label_score["score"]        
        mapped_label = label_map.get(raw_label, raw_label)
        row[mapped_label] = score
    all_scores_en.append(row)

results_en_df = pd.DataFrame(all_scores_en)

all_scores_fr = []
for doc_result in results_fr:
    row = {"negative": 0.0, "neutral": 0.0, "positive": 0.0}
    for label_score in doc_result:
        raw_label = label_score["label"]
        score = label_score["score"]
        mapped_label = label_map.get(raw_label, raw_label)
        row[mapped_label] = score
    all_scores_fr.append(row)

results_fr_df = pd.DataFrame(all_scores_fr)


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu
c:\Users\irene\anaconda3\envs\sent\Lib\site-packages\transformers\pipelines\text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead 

In [34]:
en_indices = data_pd.index[
    (data_pd['language'] == 'en') & (data_pd['ResponseM'].notna())
]
fr_indices = data_pd.index[
    (data_pd['language'] == 'fr') & (data_pd['ResponseM'].notna())
]

data_pd['negative'] = pd.NA
data_pd['neutral'] = pd.NA
data_pd['positive'] = pd.NA

data_pd.loc[en_indices, 'negative'] = results_en_df['negative'].values
data_pd.loc[en_indices, 'neutral']  = results_en_df['neutral'].values
data_pd.loc[en_indices, 'positive'] = results_en_df['positive'].values
data_pd.loc[fr_indices, 'negative'] = results_fr_df['negative'].values
data_pd.loc[fr_indices, 'neutral']  = results_fr_df['neutral'].values
data_pd.loc[fr_indices, 'positive'] = results_fr_df['positive'].values


data_pd


,Question,ResponseM,pid,language,negative,neutral,positive
0,2,protecting the elderly protecting children rig...,1890243c-fec2-eb11-a7ad-28187818aecc,en,0.050056,0.771526,0.178418
1,2,"amnesty international freedom of opinions, pre...",62e4e4eb-9bb9-eb11-a7ad-28187818aecc,en,0.031878,0.566977,0.401145
2,2,liberté droit d'un individus droit familial,7b8260c7-9bb9-eb11-a7ad-28187818aecc,fr,0.293362,0.511454,0.195184
3,1,family nature canadian rights freedom financial,9e78e854-9bb9-eb11-a7ad-28187818aecc,en,0.035859,0.783253,0.180888
4,2,liberté respecter les autres manger a sa faim ...,31d268bb-9bb9-eb11-a7ad-28187818aecc,fr,0.152161,0.327354,0.520485
...,...,...,...,...,...,...,...
877,1,nothing the outdoors not indigenous peoples,0461f6af-a1c3-eb11-a7ad-28187818aecc,en,0.612219,0.370389,0.017392
878,1,anglais hydro sirop forêts hiver autochtones,5094395e-eebe-eb11-a7ad-28187818aecc,fr,0.290965,0.581333,0.127702
879,2,certain parties support keeping human rights f...,bc534d7e-fdc2-eb11-a7ad-28187818aecc,en,0.253142,0.725021,0.021837
880,2,equality justice conflict going too far with n...,116391bc-eebe-eb11-a7ad-28187818aecc,en,0.548474,0.432982,0.018544


In [54]:
import numpy as np

threshold = 0.1

data_pd = data_pd.dropna(subset=['negative', 'neutral', 'positive']).reset_index(drop=True)

def is_uncertain(row, threshold):
    values = [row['negative'], row['neutral'], row['positive']]
    sorted_values = sorted(values, reverse=True)
    return (sorted_values[0] - sorted_values[1]) < threshold

data_pd['is_uncertain'] = data_pd.apply(lambda row: is_uncertain(row, threshold), axis=1)


In [55]:
data_pd

,Question,ResponseM,pid,language,negative,neutral,positive,is_uncertain
0,2,protecting the elderly protecting children rig...,1890243c-fec2-eb11-a7ad-28187818aecc,en,0.050056,0.771526,0.178418,False
1,2,"amnesty international freedom of opinions, pre...",62e4e4eb-9bb9-eb11-a7ad-28187818aecc,en,0.031878,0.566977,0.401145,False
2,2,liberté droit d'un individus droit familial,7b8260c7-9bb9-eb11-a7ad-28187818aecc,fr,0.293362,0.511454,0.195184,False
3,1,family nature canadian rights freedom financial,9e78e854-9bb9-eb11-a7ad-28187818aecc,en,0.035859,0.783253,0.180888,False
4,2,liberté respecter les autres manger a sa faim ...,31d268bb-9bb9-eb11-a7ad-28187818aecc,fr,0.152161,0.327354,0.520485,False
...,...,...,...,...,...,...,...,...
876,1,nothing the outdoors not indigenous peoples,0461f6af-a1c3-eb11-a7ad-28187818aecc,en,0.612219,0.370389,0.017392,False
877,1,anglais hydro sirop forêts hiver autochtones,5094395e-eebe-eb11-a7ad-28187818aecc,fr,0.290965,0.581333,0.127702,False
878,2,certain parties support keeping human rights f...,bc534d7e-fdc2-eb11-a7ad-28187818aecc,en,0.253142,0.725021,0.021837,False
879,2,equality justice conflict going too far with n...,116391bc-eebe-eb11-a7ad-28187818aecc,en,0.548474,0.432982,0.018544,False


In [56]:
data_true = data_pd[data_pd['is_uncertain'] == True]
print(len(data_true))

data_false = data_pd[data_pd['is_uncertain'] == False]
print(len(data_false))

160
721


In [61]:
print(data_pd.iloc[880].to_dict())

{'Question': 1, 'ResponseM': 'liberalism pro abortion pro gay marriage pro transexual multicultural authoritarian', 'pid': 'fae1254c-eebe-eb11-a7ad-28187818aecc', 'language': 'en', 'negative': 0.5220577120780945, 'neutral': 0.44449731707572937, 'positive': 0.03344495967030525, 'is_uncertain': True}
